In [ ]:
class ImageProcessorFactory:
    def __init__(self, pixels, width, height):
        self.pixels = pixels
        self.width = width
        self.height = height

    def get_pixel(self, x, y):
        return self.pixels[y * self.width + x]

    def set_pixel(self, x, y, pixel):
        self.pixels[y * self.width + x] = pixel

    def to_grayscale(self):
        for i in range(len(self.pixels)):
            r, g, b = self.pixels[i]
            gray = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            self.pixels[i] = (gray, gray, gray)
        return self

    def apply_blur(self):
        new_pixels = [(0, 0, 0)] * len(self.pixels)

        for y in range(1, self.height - 1):
            for x in range(1, self.width - 1):
                r_sum = g_sum = b_sum = 0
                count = 0

                for j in range(-1, 2):
                    for i in range(-1, 2):
                        r, g, b = self.get_pixel(x + i, y + j)
                        r_sum += r
                        g_sum += g
                        b_sum += b
                        count += 1

                new_pixel = (r_sum // count, g_sum // count, b_sum // count)
                self.set_pixel(x, y, new_pixel)

        self.pixels = new_pixels
        return self

    def invert_colors(self):
        for i in range(len(self.pixels)):
            r, g, b = self.pixels[i]
            self.pixels[i] = (255 - r, 255 - g, 255 - b)
        return self

    def get_pixels(self):
        return self.pixels

In [ ]:
# Read the image data
input_image_path = "input_image.bmp"
header, data, width, height, bits_per_pixel = read_image(input_image_path)

# Create the factory object with the pixel data
image_processor = ImageProcessorFactory(pixels, width, height)

# Apply desired effects
image_processor.to_grayscale()
image_processor.apply_blur()
image_processor.invert_colors()

# Get the modified pixel data
enhanced_pixels = image_processor.get_pixels()

# Convert the pixel data to bytes
enhanced_data = b''
for pixel in enhanced_pixels:
    enhanced_data += bytes(pixel)

# Save the enhanced image to a new file
output_image_path = "output_image.bmp"
write_image(output_image_path, header, enhanced_data)